In [ ]:
from zscore_strategy import DynamicZScoreStrategy
from datetime import datetime, timedelta

# 设置时间范围(获取最近约1个月的数据)
end_date = datetime.now().strftime("%Y-%m-%d 15:05:00")
start_date = (datetime.now() - timedelta(days=12)).strftime("%Y-%m-%d 09:25:00")

# 创建策略实例
strategy = DynamicZScoreStrategy(
    symbol="512800"
)

# 获取数据
strategy.fetch_data(start_date, end_date)

# 计算Z-Score
strategy.calculate_zscore()

# 计算动态阈值
strategy.calculate_dynamic_thresholds()

# 生成交易信号
strategy.generate_signals()

# 回测策略
trade_history, final_value, strategy_return, benchmark_return, max_drawdown = strategy.backtest()

# 可视化结果
strategy.plot_results()

In [ ]:
strategy.data